# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd

df = pd.read_csv('homepage_actions.csv')

num_users = df['id'].nunique()
num_users

6328

In [2]:
duplicates = df[df['id'].duplicated()]
duplicates

,timestamp,id,group,action
8,2016-09-24 20:58:01.948663,349125,experiment,click
12,2016-09-24 21:06:27.553057,601714,experiment,click
15,2016-09-24 21:30:02.739756,487634,experiment,click
17,2016-09-24 23:01:12.108316,468601,experiment,click
21,2016-09-25 00:01:47.933853,555973,experiment,click
...,...,...,...,...
8162,2017-01-17 23:20:35.483601,451198,control,click
8164,2017-01-17 23:47:58.209653,252195,control,click
8167,2017-01-18 00:56:24.554729,344770,experiment,click
8180,2017-01-18 08:54:56.879682,615849,experiment,click


In [3]:
viewers = df[df['action'] == 'view']['id'].nunique()
print(f"viewers: {viewers}")

clickers = df[df['action'] == 'click']['id'].nunique()
print(f"clickers: {clickers}")

click_no_view = df[~df['id'].isin(df[df['action'] == 'view']['id']) & (df['action'] == 'click')]
print(f"users who clicked but didn't view: {len(click_no_view)}")

viewers: 6328
clickers: 1860
users who clicked but didn't view: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [4]:
# calculate views and clicks for different groups
control_viewer_count = df[(df['group'] == 'control') & (df['action'] == 'view')]['id'].nunique()
control_clicker_count = df[(df['group'] == 'control') & (df['action'] == 'click')]['id'].nunique()

experiment_viewer_count = df[(df['group'] == 'experiment') & (df['action'] == 'view')]['id'].nunique()
experiment_clicker_count = df[(df['group'] == 'experiment') & (df['action'] == 'click')]['id'].nunique()

print(f"control_viewer_count: {control_viewer_count}")
print(f"control_clicker_count: {control_clicker_count}")
print(f"experiment_viewer_count: {experiment_viewer_count}")
print(f"experiment_clicker_count: {experiment_clicker_count}")

control_viewer_count: 3332
control_clicker_count: 932
experiment_viewer_count: 2996
experiment_clicker_count: 928


In [5]:
# calculate conversion rates

control_conversion_rate = control_clicker_count / control_viewer_count
experiment_conversion_rate = experiment_clicker_count / experiment_viewer_count

print(f"control_conversion_rate: {control_conversion_rate}")
print(f"experiment_conversion_rate: {experiment_conversion_rate}")

control_conversion_rate: 0.2797118847539016
experiment_conversion_rate: 0.3097463284379172


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [6]:
# z stat calculated below!

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [7]:
from statsmodels.stats.proportion import proportions_ztest

count = [control_clicker_count, experiment_clicker_count]
nobs = [control_viewer_count, experiment_viewer_count]

z_stat, p_value = proportions_ztest(count, nobs)
print(f"Z-statistic: {z_stat:.3f}")
print(f"P-value: {p_value:.3f}")

Z-statistic: -2.619
P-value: 0.009


### Analysis:

Does this result roughly match that of the previous statistical test?

> The z-statistic (-2.619) represents the number of standard deviations the observed difference in conversion rates is 
from the null hypothesis. A negative z-statistic tells us that the conversion rate is lower than the control group's.

> The P-value (0.009) is less than the significance level (0.05), which means we reject the null hypothesis. This means there
was a statistically significant difference between the conversion rates of the control and experiment groups.

> However, with a negative z-statistic, this means the experimental homepage performed poorer than the control homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.